## Advanced Lane Finding Project

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

---
## First, I'll compute the camera calibration using chessboard images

In [1]:
import numpy as np
import cv2
import glob
import os
import matplotlib.pyplot as plt
import pickle
%matplotlib qt
%matplotlib inline

'''
Camera class is used to:
    - calculate calibration parameters for the camera based on a set of calibration images
    - store calibration parameters for later use
    - read in calibration parameters to save on processing time
    - undistort an image using calibration parameters, for later processing
    - undistort a set of test images using calibration parameters, for demo purposes
'''
class Camera():
    def __init__(self):
        # camera calibration parameters
        self.mtx = []
        self.dist = []

    # calculate camera calibration parameters based on the calibration images
    def calculate_calibration_parameters(self):
        # prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
        objp = np.zeros((6*9,3), np.float32)
        objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

        # Arrays to store object points and image points from all the images.
        objpoints = [] # 3d points in real world space
        imgpoints = [] # 2d points in image plane.

        # Make a list of calibration images
        images = glob.glob('camera_cal/calibration*.jpg')

        # Step through the list and search for chessboard corners
        for fname in images:
            img = cv2.imread(fname)
            gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
            img_size = (img.shape[1], img.shape[0])

            # Find the chessboard corners
            ret, corners = cv2.findChessboardCorners(gray, (9,6),None)

            # If found, add object points, image points
            if ret == True:
                objpoints.append(objp)
                imgpoints.append(corners)

                # Draw and display the corners
                img = cv2.drawChessboardCorners(img, (9,6), corners, ret)
                cv2.imshow('img',img)
                cv2.waitKey(500)

        cv2.destroyAllWindows()

        if len(objpoints) & len(imgpoints):
            # Do camera calibration given object points and image points
            ret, self.mtx, self.dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img_size, None, None)

        # Save the camera calibration result for later use (we won't worry about rvecs / tvecs)
        dist_pickle = {}
        dist_pickle["mtx"] = self.mtx
        dist_pickle["dist"] = self.dist
        pickle.dump( dist_pickle, open( "camera_cal/camera_calibration_parameters.p", "wb" ) )

        print("Created new calibration parameters: mtx=", self.mtx, "; dist=", self.dist)


    # Read in the saved calibration parameters
    def read_calibration_parameters(self):
        calibration_file_path = "camera_cal/camera_calibration_parameters.p"
        if os.path.isfile(calibration_file_path):
            dist_pickle = pickle.load( open( calibration_file_path, "rb" ) )
            self.mtx = dist_pickle["mtx"]
            self.dist = dist_pickle["dist"]

            print("Loaded calibration parameters: mtx=", self.mtx, "; dist=", self.dist)


    # read saved calibration parameters or calculate based on the calibration images
    def initialize_calibration_parameters(self):
        # try to read calibration parameters saved before (speed up processing during development)
        self.read_calibration_parameters()

        # if saved camera calibration parameters were not found - try to calibrate camera
        if not (len(self.mtx) & len(self.dist)):
            self.calculate_calibration_parameters()


    # if camera calibration parameters are initiaized properly - undistort test images
    def undistort_test_images(self):
        if len(self.mtx) & len(self.dist):

            in_folder = "test_images/"
            out_folder = "output_images/"

            if not os.path.exists(out_folder):
                os.makedirs(out_folder)

            source_files = os.listdir(in_folder)

            # Read sample images, undistort and write to the output folder
            for source_file in source_files:

                source_path = in_folder + source_file
                img = cv2.imread(source_path)

                undistorted = cv2.undistort(img, self.mtx, self.dist, None, self.mtx)

                result_path = out_folder + source_file
                print("result_path=", result_path);
                cv2.imwrite(result_path, undistorted)

                # Draw and display undistorted image
                f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
                f.tight_layout()
                ax1.imshow(cv2.cvtColor(img,cv2.COLOR_BGR2RGB))
                ax1.set_title('Original Image', fontsize=50)
                ax2.imshow(cv2.cvtColor(undistorted,cv2.COLOR_BGR2RGB))
                ax2.set_title('Undistorted Image', fontsize=50)
                plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)
                plt.show()

    # undistort image using the camera calibration parameters
    def undistort_image(self, image):
        if len(self.mtx) & len(self.dist):
            return cv2.undistort(image, self.mtx, self.dist, None, self.mtx)
                



## And so on and so forth...

In [2]:
def debug_show_images(img1, title1, img2, title2, cmap1=None, cmap2=None):
    # Plot the result
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
    f.tight_layout()
    ax1.imshow(img1, cmap=cmap1)
    ax1.set_title(title1, fontsize=50)
    ax1.tick_params(labelbottom=True, labeltop=True, labelleft=True, labelright=True,
                     bottom=True, top=True, left=True, right=True)
    ax2.imshow(img2, cmap=cmap2)
    ax2.set_title(title2, fontsize=50)
    ax2.tick_params(labelbottom=True, labeltop=True, labelleft=True, labelright=True,
                     bottom=True, top=True, left=True, right=True)
    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)
    plt.show()


In [3]:
class BinaryImage():
    def __init__(self):
        pass

In [4]:
# Calculate directional gradient and apply threshold
def abs_sobel_threshold(gray, orient='x', sobel_kernel=3, thresh=(0, 255)):
    
    # Apply the following steps to img
    # 1) Take the derivative in x or y given orient = 'x' or 'y'
    # 2) Take the absolute value of the derivative or gradient
    # 3) Scale to 8-bit (0 - 255) then convert to type = np.uint8
    # 4) Create a mask of 1's where the scaled gradient magnitude 
            # is > thresh_min and < thresh_max
    # 5) Return this mask as your binary_output image
    
    # derivative
    if orient == 'x':
        sobel = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    else:
        sobel = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    
    # absolute value
    abs_sobel = np.absolute(sobel)
    
    # scale to 0-255
    abs_sobel_max = np.max(abs_sobel)
    abs_sobel = np.uint8(255*abs_sobel/abs_sobel_max)
    
    # apply threshold
    binary_output = np.zeros_like(abs_sobel)
    binary_output[(abs_sobel >= thresh[0]) & (abs_sobel <= thresh[1])] = 1
    
    # display the result
    #debug_show_images(image, 'Original Image', binary_output, 'Thresholded Gradient by ' + orient, cmap2 = 'gray')
    
    return binary_output

# Calculate gradient magnitude and apply threshold
def mag_threshold(gray, sobel_kernel=3, mag_thresh=(0, 255)):

    # Apply the following steps to img
    # 1) Take the gradient in x and y separately
    # 2) Calculate the magnitude 
    # 3) Scale to 8-bit (0 - 255) and convert to type = np.uint8
    # 4) Create a binary mask where mag thresholds are met
    # 5) Return this mask as your binary_output image
    
    # derivative
    sobel_x = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobel_y = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    
    # magnitude
    sobel = np.zeros_like(sobel_x)
    sobel = (sobel_x**2 + sobel_y**2)**(1/2.0)
    
    # scale to 0-255
    sobel_max = np.max(sobel)
    sobel = np.uint8(255*sobel/sobel_max)
    
    # apply threshold
    binary_output = np.zeros_like(sobel)
    binary_output[(sobel >= mag_thresh[0]) & (sobel <= mag_thresh[1])] = 1

    # display the result
    #debug_show_images(image, 'Original Image', binary_output, 'Thresholded Magnitude', cmap2 = 'gray')
    
    return binary_output

# Calculate gradient direction and apply threshold
def dir_threshold(gray, sobel_kernel=3, thresh=(0, np.pi/2)):
    
    # Apply the following steps to img
    # 1) Take the gradient in x and y separately
    # 2) Take the absolute value of the x and y gradients
    # 3) Use np.arctan2(abs_sobely, abs_sobelx) to calculate the direction of the gradient 
    # 4) Create a binary mask where direction thresholds are met
    # 5) Return this mask as your binary_output image
    
    # derivative
    sobel_x = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobel_y = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    
    # absolute value and direction of the gradient
    grad_direction = np.arctan2(np.absolute(sobel_y), np.absolute(sobel_x))
    
    # apply threshold
    binary_output = np.zeros_like(grad_direction)
    binary_output[(grad_direction >= thresh[0]) & (grad_direction <= thresh[1])] = 1
    
    # display the result
    #debug_show_images(image, 'Original Image', binary_output, 'Thresholded Grad. Dir.', cmap2 = 'gray')
    
    return binary_output


# calculate binary image based on different gradient thresholds (absolute, magnitude, direction)
def gradient_threshold(image):
    
    # each of the thresholding functions works with grayscale image
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    
    # Apply each of the thresholding functions
    gradx = abs_sobel_threshold(gray, orient='x', sobel_kernel=3, thresh=(20, 100))
    grady = abs_sobel_threshold(gray, orient='y', sobel_kernel=3, thresh=(20, 100))
    mag_binary = mag_threshold(gray, sobel_kernel=9, mag_thresh=(30, 100))
    dir_binary = dir_threshold(gray, sobel_kernel=15, thresh=(0.7, 1.3))

    combined = np.zeros_like(dir_binary)
    combined[((gradx == 1) & (grady == 1)) | ((mag_binary == 1) & (dir_binary == 1))] = 1
    
    # display the result
    #debug_show_images(image, 'Original Image', combined, 'Combined Grad.-Thresh. Image', cmap2 = 'gray')
    
    return combined


In [5]:
def colour_threshold(image, s_thresh=(170, 255), sx_thresh=(20, 100)):
    #image = np.copy(image)
    
    # Convert to HLS color space and separate the V channel
    hls = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
    l_channel = hls[:,:,1]
    s_channel = hls[:,:,2]
    # Sobel x on luminance channel
    sobelx = cv2.Sobel(l_channel, cv2.CV_64F, 1, 0) # Take the derivative in x
    abs_sobelx = np.absolute(sobelx) # Absolute x derivative to accentuate lines away from horizontal
    scaled_sobel = np.uint8(255*abs_sobelx/np.max(abs_sobelx))
    
    # Threshold x gradient
    sxbinary = np.zeros_like(scaled_sobel)
    sxbinary[(scaled_sobel >= sx_thresh[0]) & (scaled_sobel <= sx_thresh[1])] = 1
    
    # Threshold color channel
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= s_thresh[0]) & (s_channel <= s_thresh[1])] = 1

    # Combine the two binary thresholds
    combined_binary = np.zeros_like(sxbinary)
    combined_binary[(s_binary == 1) | (sxbinary == 1)] = 1
    return combined_binary
    
    # Stack each channel
    color_binary = np.dstack(( np.zeros_like(sxbinary), sxbinary, s_binary)) * 255
    return color_binary


In [6]:
def combined_threshold(gradient_binary, colour_binary):
    
    # Combine the two binary thresholds
    combined_binary = np.zeros_like(gradient_binary)
    combined_binary[(gradient_binary == 1) | (colour_binary == 1)] = 1
    #combined_binary[(colour_binary == 1)] = 1
    return combined_binary


In [7]:
class PerspectiveTransform():
    def __init__(self, xsize, ysize):

        self.xsize = xsize
        self.ysize = ysize
        
        # define region of interest
        near_distance = ysize-1
        far_distance = ysize*0.645

        left_bottom = [0+70, near_distance]
        right_bottom = [xsize-1-70, near_distance]

        left_apex = [xsize/2-89, far_distance]
        right_apex = [xsize/2-1+89, far_distance]

        src = np.float32(
            [left_bottom,
            left_apex,
            right_apex,
            right_bottom])
        dst = np.float32(
            [[0, ysize-1],
            [0, 0],
            [xsize-1, 0],
            [xsize-1, ysize-1]])

        # calculate perspective matrix and inverse perspective matrix
        self.M = cv2.getPerspectiveTransform(src, dst)
        self.Minv = cv2.getPerspectiveTransform(dst, src)
        
    def warp_to_birds_eye_view(self, image):
        # draw source region into the original image
        # for debug purposes only!
        # must be disabled after the region is set, otherwise the detection pipeline works incorrectly!
        #for i in range(0, len(src)-1):
        #    cv2.line(image, (src[i][0], src[i][1]), (src[i+1][0], src[i+1][1]), [255, 0, 0], 2)
        return cv2.warpPerspective(image, self.M, (self.xsize, self.ysize), flags=cv2.INTER_LINEAR)
    
    def warp_to_perspective(self, image):
        return cv2.warpPerspective(image, self.Minv, (self.xsize, self.ysize), flags=cv2.INTER_LINEAR)


In [77]:
# Define a class to receive the characteristics of each line detection
class Line():
    
    def __init__(self, side = "left"):
        
        # which side of  the lane the line is on
        # used to detect initial line position on the histogram and for different colouring
        self.side = side
        
        # was the line detected in the last iteration?
        self.detected = False  
        
        # x values of the last n fits of the line
        self.recent_xfitted = [] 
        
        #average x values of the fitted line over the last n iterations
        self.bestx = None     
        
        #polynomial coefficients averaged over the last n iterations
        self.best_fit = None  
        
        #polynomial coefficients for the most recent fit
        self.current_fit = [np.array([False])]
        
        # number of previous detections to keep for stabilization
        self.stabilization_history = 5
        
        #radius of curvature of the line in some units
        self.radius_of_curvature = None 
        
        #distance in meters of vehicle center from the line
        self.line_base_pos = None 
        
        #difference in fit coefficients between last and new fits
        self.diffs = np.array([0,0,0], dtype='float') 
        
        #x values for detected line pixels
        self.allx = None
        
        #y values for detected line pixels
        self.ally = None
        
        # Parameters for the sliding window search
        # The threshold for the histogram value
        self.histogram_threshold = 50
        # The number of sliding windows
        self.nwindows = 9
        # The width of the windows +/- margin
        self.margin_sliding_window = 100
        # Minimum number of pixels found to recenter window
        self.minpix = 50
        
        # Parameters for the search around polynomial
        # The width of the margin around the previous polynomial to search
        self.margin_polynomial_search = 100
        
        self.detection_confidence = 0
        
        # Define conversions in x and y from pixels space to meters
        self.ym_per_pix = 27/720 # meters per pixel in y dimension
        self.xm_per_pix = 3.7/1000 # meters per pixel in x dimension


        
    def measure_curvature_pixels(self, y_eval):
        '''
        Calculates the curvature of polynomial functions in pixels.
        '''
        curverad = 0
        
        if self.detected:
            # Calculate radius of curvature
            curverad = ((1 + (2*self.best_fit[0]*y_eval + self.best_fit[1])**2)**1.5) / np.abs(2*self.best_fit[0])

        return curverad

    def measure_curvature_real(self, y_eval):
        '''
        Calculates the curvature of polynomial functions in meters.
        '''
        self.radius_of_curvature = 0

        if self.detected:
            y = y_eval * self.ym_per_pix
            a = self.best_fit[0] * self.xm_per_pix / (self.ym_per_pix ** 2)
            b = self.best_fit[1] * (self.xm_per_pix / self.ym_per_pix)

            # Calculate radius of curvature
            self.radius_of_curvature = ((1 + (2*a*y + b)**2)**1.5) / np.abs(2*a)

        return self.radius_of_curvature

    # find lane pixels and color them
    def find_lane_pixels_histohramm_sliding_window(self, binary_warped, out_img):

        # Take a histogram of the bottom half of the image
        histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)
        #plt.plot(histogram)
        #plt.show()
        # Find the peak of the left or right halves of the histogram (depending on the initial parameter 'side')
        # These will be the starting point for the left or right lines
        midpoint = np.int(histogram.shape[0]//2)
        x_base = np.argmax(histogram[:midpoint]) if self.side == "left" else np.argmax(histogram[midpoint:]) + midpoint
        #print("side=", self.side, "; midpoint=", midpoint, "; x_base=", x_base, "; histogram[x_base]=", histogram[x_base])
        
        # check if the histogram values make sense
        if histogram[x_base] < self.histogram_threshold:
            self.detected = False
            return [], []

        # Set height of windows - based on nwindows above and image shape
        window_height = np.int(binary_warped.shape[0]//self.nwindows)
        # Identify the x and y positions of all nonzero pixels in the image
        nonzero = binary_warped.nonzero()
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])
        #print(nonzeroy, nonzerox)
        # Current positions to be updated later for each window in nwindows
        x_current = x_base

        # Create empty lists to receive lane pixel indices
        lane_inds = []

        # Step through the windows one by one
        for window in range(self.nwindows):
            # Identify window boundaries in x and y (and right and left)
            win_y_low = binary_warped.shape[0] - (window+1)*window_height
            win_y_high = binary_warped.shape[0] - window*window_height
            ### Find the four below boundaries of the window ###
            win_x_low = x_current - self.margin_sliding_window
            win_x_high = x_current + self.margin_sliding_window
            #print("win_y_low=", win_y_low, "win_y_high=", win_y_high, "win_x_low=", win_x_low, "win_x_high=", win_x_high)

            # Draw the window on the visualization image
            cv2.rectangle(out_img, (win_x_low,win_y_low), (win_x_high,win_y_high), (0,255,0), 2) 

            ### Identify the nonzero pixels in x and y within the window ###
            #print(len(nonzeroy), len(nonzerox))
            good_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) &
                    (nonzerox >= win_x_low) & (nonzerox < win_x_high)).nonzero()[0]

            # Append these indices to the lists
            lane_inds.append(good_inds)

            ### If you found > minpix pixels, recenter next window `x_current` on their mean position ###
            #print(len(good_inds))
            if len(good_inds) > self.minpix:
                x_current = np.int(np.mean(nonzerox[good_inds]))

        # Concatenate the arrays of indices (previously was a list of lists of pixels)
        try:
            lane_inds = np.concatenate(lane_inds)
        except ValueError:
            # Avoids an error if the above is not implemented fully
            pass

        # Extract left and right line pixel positions
        x = nonzerox[lane_inds]
        y = nonzeroy[lane_inds]
        
        self.detection_confidence = len(lane_inds)

        ## Visualization ##
        # Color the left and right lane regions
        out_img[y, x] = [255, 0, 0] if self.side == "left" else [0, 0, 255]

        self.detected = True
        return x, y

    # find lane pixels and color them
    def find_lane_pixels_search_around_polynomial(self, binary_warped, out_img):

        # check if there is a previous polynomial detection available
        if len(self.best_fit) < 3:
            self.detected = False
            return [], []

        # Grab activated pixels
        nonzero = binary_warped.nonzero()
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])

        ### Set the area of search based on activated x-values ###
        ### within the +/- margin of our polynomial function ###
        ploty = nonzeroy
        fitx = self.best_fit[0]*ploty**2 + self.best_fit[1]*ploty + self.best_fit[2]
        lane_inds = ((nonzerox >= (fitx-self.margin_polynomial_search)) & (nonzerox < (fitx+self.margin_polynomial_search)))
        
        self.detection_confidence = len(lane_inds)

        # Again, extract left and right line pixel positions
        x = nonzerox[lane_inds]
        y = nonzeroy[lane_inds] 

        ## Visualization ##
        # Color the lane pixels
        out_img[y, x] = [255, 0, 0] if self.side == "left" else [0, 0, 255]
        # Generate two lines to mark polygon for illustration of the search window area
        # Recast the x and y points into usable format for cv2.fillPoly()
        # And draw the lines onto the warped image
        # on the left
        line_window1 = np.array([np.transpose(np.vstack([fitx-self.margin_polynomial_search-2, ploty]))])
        line_window2 = np.array([np.flipud(np.transpose(np.vstack([fitx-self.margin_polynomial_search+2, ploty])))])
        line_pts = np.hstack((line_window1, line_window2))
        cv2.fillPoly(out_img, np.int_([line_pts]), (0,255, 0))
        # on the right
        line_window1 = np.array([np.transpose(np.vstack([fitx+self.margin_polynomial_search-2, ploty]))])
        line_window2 = np.array([np.flipud(np.transpose(np.vstack([fitx+self.margin_polynomial_search+2, ploty])))])
        line_pts = np.hstack((line_window1, line_window2))
        cv2.fillPoly(out_img, np.int_([line_pts]), (0,255, 0))
        
        self.detected = True
        return x, y

    def fit_polynomial(self, img_shape, x, y):

        ### Fit a second order polynomial to each using `np.polyfit` ###
        fit = np.polyfit(y, x, 2)
        #print(x, y, fit)

        # Generate x and y values for plotting
        self.ally = np.linspace(0, img_shape[0]-1, img_shape[0] )
        try:
            self.allx = fit[0]*self.ally**2 + fit[1]*self.ally + fit[2]
            # add new polynomial detection into the stabilization history list
            self.current_fit.insert(0, fit)
            # trim stabilization history list to the configured size
            if(len(self.current_fit) > self.stabilization_history):
                self.current_fit.pop()
            # update best fit polynomial parameters
            self.best_fit = fit
            # flag successful detection
            self.detected = True
        except TypeError:
            # Avoids an error if `fit` are still none or incorrect
            print('The function failed to fit a line!')
            self.allx = 1*self.ally**2 + 1*self.ally
            self.detected = False

        #print("self.allx=", self.allx, "; self.ally=", self.ally)

    def visualize_lines(self, out_img):
        # Plot the polynomial lines onto the image
        # Generate a polygon to illustrate the polinomial found
        # And recast the x and y points into usable format for cv2.fillPoly()
        line_window1 = np.array([np.transpose(np.vstack([self.allx-2, self.ally]))])
        line_window2 = np.array([np.flipud(np.transpose(np.vstack([self.allx+2, self.ally])))])
        line_pts = np.hstack((line_window1, line_window2))

        # Draw the lane onto the warped blank image
        cv2.fillPoly(out_img, np.int_([line_pts]), (255,255, 0))

    def find_line(self, binary_warped, out_img):

        # if lane was detected in the previous image - try to search around polynomial
        if self.detected:
            
            # find pixels along the polynomial
            x, y = self.find_lane_pixels_search_around_polynomial(binary_warped, out_img)
            
            # if pixels were found
            if self.detected:
                
                # fit polynomial
                self.fit_polynomial(binary_warped.shape, x, y)
            
        # if there's no detection from the previous frame OR detection based on the previous frame failed - try sliding window
        if not self.detected:
            
            # find pixels using histogram and sliding window approach
            x, y = self.find_lane_pixels_histohramm_sliding_window(binary_warped, out_img)

            # if pixels were found
            if self.detected:
                
                # fit polynomial
                self.fit_polynomial(binary_warped.shape, x, y)

        # if detection is successful - then measure and display curvature
        if self.detected:
        
            # visualize polynomial
            self.visualize_lines(out_img)
            
            # Define y-value where we want radius of curvature
            # We'll choose the maximum y-value, corresponding to the bottom of the image
            y_eval = np.max(self.ally)

            # measure and display curvature radius in pixels
            #curvature_px = self.measure_curvature_pixels(y_eval)
            #print('curvature radius = ', curvature_px, 'px')

            # calculate and display curvature radius in meters
            y = y_eval * self.ym_per_pix
            a = self.best_fit[0] * self.xm_per_pix / (self.ym_per_pix ** 2)
            b = self.best_fit[1] * (self.xm_per_pix / self.ym_per_pix)
            self.radius_of_curvature = ((1 + (2*a*y + b)**2)**1.5) / np.abs(2*a)
            #print('self.radius_of_curvature = ', self.radius_of_curvature, 'm')
            
            # calculate distance from the middle of the car to the line
            midpoint = binary_warped.shape[1]//2
            line_point = self.allx[-1]
            self.line_base_pos = abs(midpoint - line_point) * self.xm_per_pix
            #print("midpoint=", midpoint, "; line_point=", line_point, "; self.line_base_pos=", self.line_base_pos)
        
        else:
            self.radius_of_curvature = 0
            self.line_base_pos = 0

        return self.detected
    

In [100]:
class Lane():
    def __init__(self):
        self.line_left = Line("left")
        self.line_right = Line("right")
        # last detected lane width
        self.lane_width = 3.7
        
    def find_lane(self, binary_warped):
        # Create an output image to draw on and visualize the result
        out_img = np.dstack((binary_warped, binary_warped, binary_warped)) * 255
        
        self.line_left.find_line(binary_warped, out_img)
        self.line_right.find_line(binary_warped, out_img)
        
        self.validate_lane()
            
        return out_img
    
    def validate_lane(self):
        # TODO:
        # 1) calculate confidence based on number of pixels used for calculation
        # 2) compare curvature radius
        # 3) introduce threshold for difference between curvature radius between lines
        # 4) if the threshold is exceeded, reject detection (set self.detected to false)
        # 5) check direction of the curvature
        # 6) introduce threshold for the curvature difference in case of different direction (may be a smaller value)
        # 7) if the line with less confidence is curved into a different direction AND the threshold is exceeded, reject detection
        if self.line_left.detected and self.line_right.detected:
            ratio = abs(self.line_left.radius_of_curvature/self.line_right.radius_of_curvature)
            if ratio < 0: ratio = 1.0/ratio 
            if ratio > 5:
                #print("line_left.radius_of_curvature=", self.line_left.radius_of_curvature, "; line_right.radius_of_curvature=", self.line_right.radius_of_curvature)
                if self.line_left.detection_confidence < self.line_right.detection_confidence:
                    self.line_left.detected = False
                    #print("Rejecting left line detection")
                else:
                    self.line_right.detected = False
                    #print("Rejecting right line detection")
            if np.sign(self.line_left.current_fit[0][0]) != np.sign(self.line_right.current_fit[0][0]):
                #print("a_left=", self.line_left.current_fit[0][0], "; a_right=", self.line_right.current_fit[0][0])
                ratio = abs(self.line_left.radius_of_curvature/self.line_right.radius_of_curvature)
                if ratio < 0: ratio = 1.0/ratio 
                if ratio > 2:
                    #print("line_left.radius_of_curvature=", self.line_left.radius_of_curvature, "; line_right.radius_of_curvature=", self.line_right.radius_of_curvature)
                    if self.line_left.detection_confidence < self.line_right.detection_confidence:
                        self.line_left.detected = False
                        #print("Rejecting left line detection")
                    else:
                        self.line_right.detected = False
                        #print("Rejecting right line detection")
        pass
        
    
    def generate_display_lane(self, binary_warped):
        
        # Create an image to draw the lines on
        warp_zero = np.zeros_like(binary_warped).astype(np.uint8)
        color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

        if self.line_left.detected and self.line_right.detected:
            # Recast the x and y points into usable format for cv2.fillPoly()
            pts_left = np.array([np.transpose(np.vstack([self.line_left.allx, self.line_left.ally]))])
            pts_right = np.array([np.flipud(np.transpose(np.vstack([self.line_right.allx, self.line_right.ally])))])
        
        # if one of the lanes is not detected - use the other's lane polynomial shifted by the last detected lane width
        else:
            if self.line_left.detected:
                pts_left = np.array([np.transpose(np.vstack([self.line_left.allx, self.line_left.ally]))])
                # substitute from the other lane
                right_allx = self.line_left.current_fit[0][0]*self.line_left.ally**2 + self.line_left.current_fit[0][1]*self.line_left.ally + self.line_left.current_fit[0][2] + self.lane_width/self.line_left.xm_per_pix
                pts_right = np.array([np.flipud(np.transpose(np.vstack([right_allx, self.line_left.ally])))])
            elif self.line_right.detected:
                pts_right = np.array([np.flipud(np.transpose(np.vstack([self.line_right.allx, self.line_right.ally])))])
                # substitute from the other lane
                left_allx = self.line_right.current_fit[0][0]*self.line_right.ally**2 + self.line_right.current_fit[0][1]*self.line_right.ally + self.line_right.current_fit[0][2] - self.lane_width/self.line_right.xm_per_pix
                pts_left = np.array([np.flipud(np.transpose(np.vstack([left_allx, self.line_right.ally])))])
            else:
                return color_warp

        pts = np.hstack((pts_left, pts_right))

        # Draw the lane onto the warped blank image
        cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))
        
        return color_warp
    
    def display_lane_properties(self, image):
        
        if self.line_left.detected and self.line_right.detected:
            
            self.lane_width = self.line_left.line_base_pos + self.line_right.line_base_pos
            lane_curvature = (self.line_left.radius_of_curvature + self.line_right.radius_of_curvature) / 2
            
            message = "Lane width is " + str(round(self.lane_width, 2)) + " m"
            cv2.putText(image, message, (10,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2)
            
            message = "Lane curvature is " + str(round(lane_curvature, 2)) + " m"
            cv2.putText(image, message, (10,100), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2)
            
            if self.line_left.line_base_pos > self.line_right.line_base_pos:
                message = "The car is " + str(round(self.line_left.line_base_pos - self.lane_width / 2, 2)) + " m to the right of the lane"
            elif self.line_left.line_base_pos < self.line_right.line_base_pos:
                message = "The car is " + str(round(self.line_right.line_base_pos - self.lane_width / 2, 2)) + " m to the left of the lane"
            else:
                message = "The car is exactly in the middle of the lane!"
            cv2.putText(image, message, (10,150), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2)
        
        else:
            message = "One of the lines was not found! Assuming lane width of " + str(round(self.lane_width, 2)) + " m"
            cv2.putText(image, message, (10,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2)
        
        


In [101]:
class ImageProcessor():
    
    def __init__(self, camera, xsize, ysize, show_debug_images = False, write_undistorted = False, write_thresholded = False, write_birds_eye_view = False, write_result = False):
        # create lane detector object
        self.camera = camera
        self.transform = PerspectiveTransform(xsize, ysize)
        self.show_debug_images = show_debug_images
        self.write_undistorted = write_undistorted
        self.write_thresholded = write_thresholded
        self.write_birds_eye_view = write_birds_eye_view
        self.write_result = write_result
        self.lane = Lane()
    
    def process_image(self, img):
        # undistort according to the camera calibration parameters
        undistorted = self.camera.undistort_image(img)
        if self.show_debug_images:
            #debug_show_images(img, 'Original Image', undistorted, 'Undistorted Image')
            pass

        # write undostorted image to output folder, if needed
        if self.write_undistorted:
            result_path = out_folder + 'undistorted_' + source_file
            print("Writing to result_path=", result_path);
            cv2.imwrite(result_path, undistorted)

        # find threholded binary image using gradients
        thresholded_grad = gradient_threshold(undistorted)
        if self.show_debug_images:
            #debug_show_images(undistorted, 'Undistorted Image', thresholded_grad, 'Thresholded by Gradients', cmap2 = 'gray')
            pass

        # another try with colour transformation and gradient
        thresholded_col = colour_threshold(undistorted)
        if self.show_debug_images:
            #debug_show_images(undistorted, 'Undistorted Image', thresholded_col, 'Thresholded by Colour', cmap2 = 'gray')
            pass

        # combine into a single threholded binary image
        combined_binary = combined_threshold(thresholded_grad, thresholded_col)
        if self.show_debug_images:
            #debug_show_images(undistorted, 'Undistorted Image', combined_binary, 'Thresholded Image', cmap2 = 'gray')
            pass

        # write threholded image to output folder, if needed
        if self.write_thresholded:
            result_path = out_folder + 'threholded_' + source_file
            print("Writing to result_path=", result_path);
            cv2.imwrite(result_path, combined_binary * 255)

        warped_image = self.transform.warp_to_birds_eye_view(undistorted)
        if self.show_debug_images:
            #debug_show_images(undistorted, 'Undistorted Image', warped_image, 'Warped Image')
            pass

        if self.write_birds_eye_view:
            result_path = out_folder + 'warped_' + source_file
            print("Writing to result_path=", result_path);
            cv2.imwrite(result_path, warped_image)

        warped_binary = self.transform.warp_to_birds_eye_view(combined_binary)
        if self.show_debug_images:
            #debug_show_images(combined_binary, 'Thresholded Image', warped_binary * 255, 'Warped Binary Image', cmap1 = 'gray', cmap2 = 'gray')
            pass

        # try to find lane in the given picture
        out_img = self.lane.find_lane(warped_binary)
        if self.show_debug_images:
            #debug_show_images(warped_binary * 255, 'Warped Binary Image', out_img.astype(int), 'Lane Detected', cmap1 = 'gray')
            pass

        # generate image with detected lane (in birds eye view)
        warped_lane_image = self.lane.generate_display_lane(warped_binary)
        # warp the image with detected lane back to original image space using inverse perspective matrix (Minv)
        perspective_lane_image = self.transform.warp_to_perspective(warped_lane_image)
        # combine the result with the original undistorted image
        result = cv2.addWeighted(undistorted, 1, perspective_lane_image, 0.3, 0)
        # write properties of the detected lane directly into the picture
        self.lane.display_lane_properties(result)
        if self.show_debug_images:
            #debug_show_images(undistorted, 'Undistorted Image', result, 'Detected Lane')
            pass

        if self.write_result:
            result_path = out_folder + 'result_' + source_file
            print("Writing to result_path=", result_path);
            cv2.imwrite(result_path, result)

        return result


In [ ]:
#
# Process pictures
#


camera = Camera()
camera.initialize_calibration_parameters()

in_folder = "test_images/"
out_folder = "output_images/"

if not os.path.exists(out_folder):
    os.makedirs(out_folder)

source_files = os.listdir(in_folder)

# Read and process sample images
for source_file in source_files:

    # read test image in
    source_path = in_folder + source_file
    print("Reading from source_path=", source_path);
    img = cv2.cvtColor(cv2.imread(source_path),cv2.COLOR_BGR2RGB)
    
    image_processor = ImageProcessor(camera, img.shape[1], img.shape[0], show_debug_images = True, write_undistorted = True, write_thresholded = True, write_birds_eye_view = True, write_result = True)
    image_processor.process_image(img)
    
    break

In [ ]:
#
# Process videos
#

# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
#from IPython.display import HTML

#input_clip = VideoFileClip("project_video.mp4").subclip(39,40)
input_clip = VideoFileClip("project_video.mp4")
output_path = "output_project_video.mp4"

# calculate region of interest
ysize = input_clip.h
xsize = input_clip.w

near_distance = ysize-1
far_distance = ysize*0.6
mid_x = (xsize-1)/2

left_bottom = [0, near_distance]
right_bottom = [xsize-1, near_distance]

left_apex = [mid_x-35, far_distance]
right_apex = [mid_x+35, far_distance]

vertices = np.array( [[left_bottom, left_apex, right_apex, right_bottom, left_bottom]], dtype=np.int32 )

# init processing pipeline with region of interest and stabilization buffer length
camera = Camera()
camera.initialize_calibration_parameters()
image_processor = ImageProcessor(camera, xsize, ysize, show_debug_images = True, write_undistorted = False, write_thresholded = False, write_birds_eye_view = False, write_result = False)

output_clip = input_clip.fl_image(image_processor.process_image) #NOTE: this function expects color images!!
%time output_clip.write_videofile(output_path, audio=False)
